# Hybrid (CNN+GRU) Classification with CR Dataset
<hr>

We will build a text classification model using CNN and GRU model on the Customer Reviews Dataset. Since there is no standard train/test split for this dataset, we will use 10-Fold Cross Validation (CV). 

The CNN model is inspired by __Yoon Kim__ paper in his study on the use of Word Embedding + CNN for text classification. The hyperparameters we use based on his study are as follows:
- Transfer function: rectified linear.
- Kernel sizes: 3, 4, 5.
- Number of filters: 100.
- Dropout rate: 0.5.
- Weight regularization (L2): 3.
- Batch Size: 50.
- Update Rule: Adam

## Load the library

In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
import nltk
import random
# from nltk.tokenize import TweetTokenizer
from sklearn.model_selection import KFold

%config IPCompleter.greedy=True
%config IPCompleter.use_jedi=False
# nltk.download('twitter_samples')

In [2]:
tf.config.list_physical_devices('GPU') 

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

## Load the Dataset

In [3]:
corpus = pd.read_pickle('../../../0_data/MR/MR.pkl')
corpus.label = corpus.label.astype(int)
print(corpus.shape)
corpus

(10662, 3)


sentence  label  split
0                       simplistic , silly and tedious .      0  train
1      it 's so laddish and juvenile , only teenage b...      0  train
2      exploitative and largely devoid of the depth o...      0  train
3      garbus discards the potential for pathological...      0  train
4      a visually flashy but narratively opaque and e...      0  train
...                                                  ...    ...    ...
10657  both exuberantly romantic and serenely melanch...      1  train
10658  mazel tov to a film about a family 's joyous l...      1  train
10659  standing in the shadows of motown is the best ...      1  train
10660  it 's nice to see piscopo again after all thes...      1  train
10661  provides a porthole into that noble , tremblin...      1  train

[10662 rows x 3 columns]

In [4]:
corpus.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10662 entries, 0 to 10661
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   sentence  10662 non-null  object
 1   label     10662 non-null  int32 
 2   split     10662 non-null  object
dtypes: int32(1), object(2)
memory usage: 208.4+ KB


In [5]:
corpus.groupby( by='label').count()

sentence  split
label                 
0          5331   5331
1          5331   5331

In [6]:
# Separate the sentences and the labels
sentences, labels = list(corpus.sentence), list(corpus.label)

In [7]:
sentences[0]

'simplistic , silly and tedious .'

<!--## Split Dataset-->

# Data Preprocessing
<hr>

Preparing data for word embedding, especially for pre-trained word embedding like Word2Vec or GloVe, __don't use standard preprocessing steps like stemming or stopword removal__. Compared to our approach on cleaning the text when doing word count based feature extraction (e.g. TFIDF) such as removing stopwords, stemming etc, now we will keep these words as we do not want to lose such information that might help the model learn better.

__Tomas Mikolov__, one of the developers of Word2Vec, in _word2vec-toolkit: google groups thread., 2015_, suggests only very minimal text cleaning is required when learning a word embedding model. Sometimes, it's good to disconnect
In short, what we will do is:
- Puntuations removal
- Lower the letter case
- Tokenization

The process above will be handled by __Tokenizer__ class in TensorFlow

- <b>One way to choose the maximum sequence length is to just pick the length of the longest sentence in the training set.</b>

In [8]:
# Define a function to compute the max length of sequence
def max_length(sequences):
    '''
    input:
        sequences: a 2D list of integer sequences
    output:
        max_length: the max length of the sequences
    '''
    max_length = 0
    for i, seq in enumerate(sequences):
        length = len(seq)
        if max_length < length:
            max_length = length
    return max_length

In [9]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

trunc_type='post'
padding_type='post'
oov_tok = "<UNK>"

print("Example of sentence: ", sentences[4])

# Cleaning and Tokenization
tokenizer = Tokenizer(oov_token=oov_tok)
tokenizer.fit_on_texts(sentences)

# Turn the text into sequence
training_sequences = tokenizer.texts_to_sequences(sentences)
max_len = max_length(training_sequences)

print('Into a sequence of int:', training_sequences[4])

# Pad the sequence to have the same size
training_padded = pad_sequences(training_sequences, maxlen=max_len, padding=padding_type, truncating=trunc_type)
print('Into a padded sequence:', training_padded[4])

Example of sentence:  a visually flashy but narratively opaque and emotionally vapid exercise in style and mystification .
Into a sequence of int: [3, 544, 1838, 13, 3909, 3366, 4, 658, 2629, 416, 10, 236, 4, 10112]
Into a padded sequence: [    3   544  1838    13  3909  3366     4   658  2629   416    10   236
     4 10112     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0]


In [10]:
word_index = tokenizer.word_index
# See the first 10 words in the vocabulary
for i, word in enumerate(word_index):
    print(word, word_index.get(word))
    if i==9:
        break
vocab_size = len(word_index)+1
print(vocab_size)

<UNK> 1
the 2
a 3
and 4
of 5
to 6
is 7
's 8
it 9
in 10
18760


# Model 1: Embedding Random
<hr>

A __standard model__ for document classification is to use (quoted from __Jason Brownlee__, the author of [machinelearningmastery.com](https://machinelearningmastery.com)):
>- Word Embedding: A distributed representation of words where different words that have a similar meaning (based on their usage) also have a similar representation.
>- Convolutional Model: A feature extraction model that learns to extract salient features from documents represented using a word embedding.
>- Fully Connected Model: The interpretation of extracted features in terms of a predictive output.


Therefore, the model is comprised of the following elements:
- __Input layer__ that defines the length of input sequences.
- __Embedding layer__ set to the size of the vocabulary and 100-dimensional real-valued representations.
- __Conv1D layer__ with 32 filters and a kernel size set to the number of words to read at once.
- __MaxPooling1D layer__ to consolidate the output from the convolutional layer.
- __Flatten layer__ to reduce the three-dimensional output to two dimensional for concatenation.

The CNN model is inspired by __Yoon Kim__ paper in his study on the use of Word Embedding + CNN for text classification. The hyperparameters we use based on his study are as follows:
- Transfer function: rectified linear.
- Kernel sizes: 3, 4, 5.
- Number of filters: 100.
- Dropout rate: 0.5.
- Weight regularization (L2): 3.
- Batch Size: 50.
- Update Rule: Adam

We will perform the best parameter using __grid search__ and 10-fold cross validation.

## CNN-GRU Model

Now, we will build Convolutional Neural Network (CNN) models to classify encoded documents as either positive or negative.

The model takes inspiration from `CNN for Sentence Classification` by *Yoon Kim*.

Now, we will define our CNN model as follows:
- One Conv layer with 100 filters, kernel size 5, and relu activation function;
- One MaxPool layer with pool size = 2;
- One Dropout layer after flattened;
- Optimizer: Adam (The best learning algorithm so far)
- Loss function: binary cross-entropy (suited for binary classification problem)

**Note**: 
- The whole purpose of dropout layers is to tackle the problem of over-fitting and to introduce generalization to the model. Hence it is advisable to keep dropout parameter near 0.5 in hidden layers. 
- https://missinglink.ai/guides/keras/keras-conv1d-working-1d-convolutional-neural-networks-keras/

In [11]:
from tensorflow.keras.utils import plot_model
from tensorflow.keras.constraints import MaxNorm
from tensorflow.keras.layers import Input, Embedding, Conv1D, Dropout, MaxPool1D, Flatten, Dense, Bidirectional, GRU
from tensorflow.keras.models import Model
from tensorflow.keras.layers import concatenate

In [12]:
def define_model(filters = 100, kernel_size = 3, activation='relu', input_dim = None, output_dim=300, max_length = None ):
  
    # Channel 1
    input1 = Input(shape=(max_length,))
    embeddding1 = Embedding(input_dim=input_dim, output_dim=output_dim, input_length=max_length)(input1)
    conv1 = Conv1D(filters=filters, kernel_size=kernel_size, activation='relu', 
                   kernel_constraint= MaxNorm( max_value=3, axis=[0,1]))(embeddding1)
    pool1 = MaxPool1D(pool_size=2, strides=2)(conv1)
    flat1 = Flatten()(pool1)
    drop1 = Dropout(0.5)(flat1)
    dense1 = Dense(10, activation='relu')(drop1)
    drop1 = Dropout(0.5)(dense1)
    out1 = Dense(1, activation='sigmoid')(drop1)
    
    # Channel 2
    input2 = Input(shape=(max_length,))
    embeddding2 = Embedding(input_dim=input_dim, output_dim=output_dim, input_length=max_length, mask_zero=True)(input2)
    gru2 = Bidirectional(GRU(64))(embeddding2)
    drop2 = Dropout(0.5)(gru2)
    out2 = Dense(1, activation='sigmoid')(drop2)
    
    # Merge
    merged = concatenate([out1, out2])
    
    # Interpretation
    outputs = Dense(1, activation='sigmoid')(merged)
    model = Model(inputs=[input1, input2], outputs=outputs)
    
    # Compile
    model.compile( loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    
    return model

In [13]:
model_0 = define_model( input_dim=1000, max_length=100)
model_0.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 100)]        0                                            
__________________________________________________________________________________________________
embedding (Embedding)           (None, 100, 300)     300000      input_1[0][0]                    
__________________________________________________________________________________________________
conv1d (Conv1D)                 (None, 98, 100)      90100       embedding[0][0]                  
__________________________________________________________________________________________________
max_pooling1d (MaxPooling1D)    (None, 49, 100)      0           conv1d[0][0]                     
______________________________________________________________________________________________

In [14]:
class myCallback(tf.keras.callbacks.Callback):
    # Overide the method on_epoch_end() for our benefit
    def on_epoch_end(self, epoch, logs={}):
        if (logs.get('accuracy') > 0.93):
            print("\nReached 93% accuracy so cancelling training!")
            self.model.stop_training=True


callbacks = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', min_delta=0, 
                                             patience=7, verbose=2, 
                                             mode='auto', restore_best_weights=True)

## Train and Test the Model

In [15]:
# Parameter Initialization
trunc_type='post'
padding_type='post'
oov_tok = "<UNK>"
activations = ['relu']
filters = 100
kernel_sizes = [1, 2, 3, 4, 5, 6]

columns = ['Activation', 'Filters', 'acc1', 'acc2', 'acc3', 'acc4', 'acc5', 'acc6', 'acc7', 'acc8', 'acc9', 'acc10', 'AVG']
record = pd.DataFrame(columns = columns)

# prepare cross validation with 10 splits and shuffle = True
kfold = KFold(10, True)

# Separate the sentences and the labels
sentences, labels = list(corpus.sentence), list(corpus.label)

for activation in activations:
    for kernel_size in kernel_sizes:
        # kfold.split() will return set indices for each split
        acc_list = []
        for train, test in kfold.split(sentences):
            
            train_x, test_x = [], []
            train_y, test_y = [], []
            
            for i in train:
                train_x.append(sentences[i])
                train_y.append(labels[i])

            for i in test:
                test_x.append(sentences[i])
                test_y.append(labels[i])

            # Turn the labels into a numpy array
            train_y = np.array(train_y)
            test_y = np.array(test_y)

            # encode data using
            # Cleaning and Tokenization
            tokenizer = Tokenizer(oov_token=oov_tok)
            tokenizer.fit_on_texts(train_x)

            # Turn the text into sequence
            training_sequences = tokenizer.texts_to_sequences(train_x)
            test_sequences = tokenizer.texts_to_sequences(test_x)

            max_len = max_length(training_sequences)

            # Pad the sequence to have the same size
            Xtrain = pad_sequences(training_sequences, maxlen=max_len, padding=padding_type, truncating=trunc_type)
            Xtest = pad_sequences(test_sequences, maxlen=max_len, padding=padding_type, truncating=trunc_type)

            word_index = tokenizer.word_index
            vocab_size = len(word_index)+1
            
            # Define the input shape
            model = define_model(filters, kernel_size, activation, input_dim=vocab_size, max_length=max_len)

            # Train the model and initialize test accuracy with 0
            acc = 0
            while(acc<0.7):
                
                # Train the model
                model.fit(x=[Xtrain, Xtrain], y = train_y, batch_size=50, epochs=100, verbose=1, 
                          callbacks=[callbacks], validation_data=([Xtest, Xtest], test_y))

                # evaluate the model
                loss, acc = model.evaluate([Xtest, Xtest], test_y, verbose=0)
                print('Test Accuracy: {}'.format(acc*100))
                
                if (acc<0.7):
                    print('The model suffered from local minimum. Retrain the model!')
                    model = define_model(filters, kernel_size, activation, input_dim=vocab_size, max_length=max_len)
                    
                else:
                    print('Done!')

            # evaluate the model
            loss, acc = model.evaluate([Xtest, Xtest], test_y, verbose=0)
            print('Test Accuracy: {}'.format(acc*100))

            acc_list.append(acc*100)
            
        mean_acc = np.array(acc_list).mean()
        parameters = [activation, kernel_size]
        entries = parameters + acc_list + [mean_acc]

        temp = pd.DataFrame([entries], columns=columns)
        record = record.append(temp, ignore_index=True)
        print()
        print(record)
        print()

C:\Users\Diardano Raihan\Anaconda3\envs\tf-gpu\lib\site-packages\sklearn\utils\validation.py:72: FutureWarning: Pass shuffle=True as keyword args. From version 1.0 (renaming of 0.25) passing these as positional arguments will result in an error
  "will result in an error", FutureWarning)


Epoch 1/100
192/192 [==============================] - 71s 244ms/step - loss: 0.6812 - accuracy: 0.5628 - val_loss: 0.6283 - val_accuracy: 0.7263
Epoch 2/100
192/192 [==============================] - 41s 215ms/step - loss: 0.5721 - accuracy: 0.8318 - val_loss: 0.5416 - val_accuracy: 0.7563
Epoch 3/100
192/192 [==============================] - 41s 214ms/step - loss: 0.4273 - accuracy: 0.9079 - val_loss: 0.5290 - val_accuracy: 0.7601
Epoch 4/100
192/192 [==============================] - 41s 212ms/step - loss: 0.3281 - accuracy: 0.9484 - val_loss: 0.5198 - val_accuracy: 0.7582
Epoch 5/100
192/192 [==============================] - 45s 234ms/step - loss: 0.2700 - accuracy: 0.9646 - val_loss: 0.5195 - val_accuracy: 0.7554
Epoch 6/100
192/192 [==============================] - 46s 239ms/step - loss: 0.2278 - accuracy: 0.9774 - val_loss: 0.5217 - val_accuracy: 0.7545
Epoch 7/100
192/192 [==============================] - 42s 217ms/step - loss: 0.1976 - accuracy: 0.9793 - val_loss: 0.5094 -

192/192 [==============================] - 51s 198ms/step - loss: 0.6990 - accuracy: 0.5560 - val_loss: 0.6648 - val_accuracy: 0.7017
Epoch 2/100
192/192 [==============================] - 34s 177ms/step - loss: 0.6334 - accuracy: 0.8284 - val_loss: 0.6400 - val_accuracy: 0.7355
Epoch 3/100
192/192 [==============================] - 34s 177ms/step - loss: 0.5862 - accuracy: 0.8826 - val_loss: 0.6287 - val_accuracy: 0.7223
Epoch 4/100
192/192 [==============================] - 34s 179ms/step - loss: 0.5421 - accuracy: 0.9093 - val_loss: 0.6202 - val_accuracy: 0.7158
Epoch 5/100
192/192 [==============================] - 34s 179ms/step - loss: 0.5023 - accuracy: 0.9256 - val_loss: 0.6093 - val_accuracy: 0.7148
Epoch 6/100
192/192 [==============================] - 34s 179ms/step - loss: 0.4628 - accuracy: 0.9465 - val_loss: 0.5910 - val_accuracy: 0.7261
Epoch 7/100
192/192 [==============================] - 34s 179ms/step - loss: 0.4302 - accuracy: 0.9615 - val_loss: 0.5855 - val_accurac

Epoch 11/100
192/192 [==============================] - 38s 199ms/step - loss: 0.2255 - accuracy: 0.9804 - val_loss: 0.5670 - val_accuracy: 0.7486
Epoch 12/100
192/192 [==============================] - 41s 213ms/step - loss: 0.2121 - accuracy: 0.9811 - val_loss: 0.5650 - val_accuracy: 0.7561
Epoch 13/100
192/192 [==============================] - 39s 203ms/step - loss: 0.1986 - accuracy: 0.9821 - val_loss: 0.5709 - val_accuracy: 0.7439
Epoch 14/100
192/192 [==============================] - 40s 207ms/step - loss: 0.1903 - accuracy: 0.9806 - val_loss: 0.5711 - val_accuracy: 0.7598
Epoch 15/100
192/192 [==============================] - 39s 202ms/step - loss: 0.1792 - accuracy: 0.9814 - val_loss: 0.5776 - val_accuracy: 0.7580
Epoch 16/100
192/192 [==============================] - 42s 220ms/step - loss: 0.1698 - accuracy: 0.9826 - val_loss: 0.5891 - val_accuracy: 0.7486
Epoch 17/100
192/192 [==============================] - 39s 204ms/step - loss: 0.1570 - accuracy: 0.9852 - val_loss: 0

Restoring model weights from the end of the best epoch.
Epoch 00009: early stopping
Test Accuracy: 76.66354179382324
Done!
Test Accuracy: 76.66354179382324
Epoch 1/100
192/192 [==============================] - 51s 200ms/step - loss: 0.6940 - accuracy: 0.5034 - val_loss: 0.6856 - val_accuracy: 0.6698
Epoch 2/100
192/192 [==============================] - 34s 176ms/step - loss: 0.6764 - accuracy: 0.7042 - val_loss: 0.6569 - val_accuracy: 0.7392
Epoch 3/100
192/192 [==============================] - 34s 175ms/step - loss: 0.6323 - accuracy: 0.8204 - val_loss: 0.6112 - val_accuracy: 0.7598
Epoch 4/100
192/192 [==============================] - 33s 174ms/step - loss: 0.5488 - accuracy: 0.8429 - val_loss: 0.5697 - val_accuracy: 0.7749
Epoch 5/100
192/192 [==============================] - 34s 175ms/step - loss: 0.4699 - accuracy: 0.9124 - val_loss: 0.5389 - val_accuracy: 0.7711
Epoch 6/100
192/192 [==============================] - 34s 175ms/step - loss: 0.3985 - accuracy: 0.9409 - val_loss

...

192/192 [==============================] - 34s 179ms/step - loss: 0.2793 - accuracy: 0.9627 - val_loss: 0.5168 - val_accuracy: 0.7683
Epoch 10/100
192/192 [==============================] - 34s 179ms/step - loss: 0.2557 - accuracy: 0.9610 - val_loss: 0.5217 - val_accuracy: 0.7655
Epoch 11/100
192/192 [==============================] - 34s 178ms/step - loss: 0.2248 - accuracy: 0.9702 - val_loss: 0.5183 - val_accuracy: 0.7692
Epoch 12/100
192/192 [==============================] - 35s 181ms/step - loss: 0.2063 - accuracy: 0.9713 - val_loss: 0.5363 - val_accuracy: 0.7617
Epoch 13/100
192/192 [==============================] - 34s 179ms/step - loss: 0.1871 - accuracy: 0.9761 - val_loss: 0.5297 - val_accuracy: 0.7636
Epoch 14/100
192/192 [==============================] - 34s 179ms/step - loss: 0.1766 - accuracy: 0.9760 - val_loss: 0.5381 - val_accuracy: 0.7674
Epoch 15/100
192/192 [==============================] - 34s 179ms/step - loss: 0.1585 - accuracy: 0.9759 - val_loss: 0.5524 - val_a

192/192 [==============================] - 36s 185ms/step - loss: 0.1075 - accuracy: 0.9857 - val_loss: 0.6477 - val_accuracy: 0.7467
Epoch 19/100
192/192 [==============================] - 35s 182ms/step - loss: 0.1009 - accuracy: 0.9858 - val_loss: 0.6625 - val_accuracy: 0.7430
Epoch 20/100
192/192 [==============================] - 35s 181ms/step - loss: 0.0954 - accuracy: 0.9848 - val_loss: 0.6674 - val_accuracy: 0.7392
Restoring model weights from the end of the best epoch.
Epoch 00020: early stopping
Test Accuracy: 75.60975551605225
Done!
Test Accuracy: 75.60975551605225
Epoch 1/100
192/192 [==============================] - 51s 200ms/step - loss: 0.6746 - accuracy: 0.5716 - val_loss: 0.6037 - val_accuracy: 0.7458
Epoch 2/100
192/192 [==============================] - 35s 180ms/step - loss: 0.5449 - accuracy: 0.8420 - val_loss: 0.5398 - val_accuracy: 0.7842
Epoch 3/100
192/192 [==============================] - 35s 180ms/step - loss: 0.4250 - accuracy: 0.9193 - val_loss: 0.5166 -

Epoch 7/100
192/192 [==============================] - 46s 240ms/step - loss: 0.2502 - accuracy: 0.9735 - val_loss: 0.5280 - val_accuracy: 0.7486
Epoch 8/100
192/192 [==============================] - 41s 214ms/step - loss: 0.2185 - accuracy: 0.9759 - val_loss: 0.5304 - val_accuracy: 0.7580
Epoch 9/100
192/192 [==============================] - 43s 222ms/step - loss: 0.1912 - accuracy: 0.9811 - val_loss: 0.5315 - val_accuracy: 0.7552
Epoch 10/100
192/192 [==============================] - 49s 254ms/step - loss: 0.1754 - accuracy: 0.9778 - val_loss: 0.5494 - val_accuracy: 0.7439
Epoch 11/100
192/192 [==============================] - 40s 206ms/step - loss: 0.1552 - accuracy: 0.9804 - val_loss: 0.5574 - val_accuracy: 0.7486
Epoch 12/100
192/192 [==============================] - 41s 214ms/step - loss: 0.1399 - accuracy: 0.9834 - val_loss: 0.5599 - val_accuracy: 0.7477
Restoring model weights from the end of the best epoch.
Epoch 00012: early stopping
Test Accuracy: 75.98499059677124
Done

192/192 [==============================] - 39s 201ms/step - loss: 0.2460 - accuracy: 0.9686 - val_loss: 0.5382 - val_accuracy: 0.7448
Epoch 8/100
192/192 [==============================] - 38s 199ms/step - loss: 0.2127 - accuracy: 0.9745 - val_loss: 0.5375 - val_accuracy: 0.7448
Epoch 9/100
192/192 [==============================] - 36s 185ms/step - loss: 0.1915 - accuracy: 0.9750 - val_loss: 0.5440 - val_accuracy: 0.7505
Epoch 10/100
192/192 [==============================] - 37s 195ms/step - loss: 0.1714 - accuracy: 0.9786 - val_loss: 0.5510 - val_accuracy: 0.7533
Restoring model weights from the end of the best epoch.
Epoch 00010: early stopping
Test Accuracy: 75.89118480682373
Done!
Test Accuracy: 75.89118480682373
Epoch 1/100
192/192 [==============================] - 56s 219ms/step - loss: 0.6894 - accuracy: 0.5484 - val_loss: 0.6326 - val_accuracy: 0.6979
Epoch 2/100
192/192 [==============================] - 39s 203ms/step - loss: 0.5529 - accuracy: 0.8369 - val_loss: 0.5958 - 

Epoch 18/100
192/192 [==============================] - 43s 225ms/step - loss: 0.1807 - accuracy: 0.9801 - val_loss: 0.5630 - val_accuracy: 0.7664
Epoch 19/100
192/192 [==============================] - 42s 220ms/step - loss: 0.1657 - accuracy: 0.9847 - val_loss: 0.5624 - val_accuracy: 0.7674
Epoch 20/100
192/192 [==============================] - 45s 232ms/step - loss: 0.1551 - accuracy: 0.9859 - val_loss: 0.5685 - val_accuracy: 0.7683
Epoch 21/100
192/192 [==============================] - 55s 285ms/step - loss: 0.1505 - accuracy: 0.9849 - val_loss: 0.5674 - val_accuracy: 0.7702
Restoring model weights from the end of the best epoch.
Epoch 00021: early stopping
Test Accuracy: 77.76735424995422
Done!
Test Accuracy: 77.76735424995422
Epoch 1/100
192/192 [==============================] - 79s 300ms/step - loss: 0.7027 - accuracy: 0.5464 - val_loss: 0.6316 - val_accuracy: 0.6839
Epoch 2/100
192/192 [==============================] - 45s 232ms/step - loss: 0.5366 - accuracy: 0.8322 - val_

## Summary

In [16]:
record.sort_values(by='AVG', ascending=False)

Activation Filters       acc1       acc2       acc3       acc4       acc5  \
0       relu       1  77.038425  78.069353  77.861166  77.485931  74.015009   
3       relu       4  78.912842  77.600747  77.954972  75.609756  74.108815   
1       relu       2  74.507964  76.663542  77.485931  76.454031  76.735461   
4       relu       5  76.101220  75.820059  77.767354  75.234520  75.609756   
5       relu       6  76.476103  75.538892  75.891185  75.984991  76.266414   
2       relu       3  77.225870  78.444237  75.984991  77.485931  75.891185   

        acc6       acc7       acc8       acc9      acc10        AVG  
0  76.923078  78.330207  75.984991  77.016884  77.579737  77.030478  
3  76.829267  79.924953  76.360226  75.609756  76.454031  76.936536  
1  77.204502  77.767354  75.891185  77.954972  78.330207  76.899515  
4  78.424013  77.204502  77.954972  75.984991  76.735461  76.683685  
5  76.266414  77.767354  74.953097  79.268295  78.048778  76.646152  
2  75.703567  74.671668  75.234520  76.547843  78.142589  76.533240

In [17]:
record[['Activation', 'AVG']].groupby(by='Activation').max().sort_values(by='AVG', ascending=False)

AVG
Activation           
relu        77.030478

In [18]:
report = record.sort_values(by='AVG', ascending=False)
report = report.to_excel('HYBRID_MR.xlsx', sheet_name='random')

# Model 2: Word2Vec Static

__Using and updating pre-trained embeddings__
* In this part, we will create an Embedding layer in Tensorflow Keras using a pre-trained word embedding called Word2Vec 300-d tht has been trained 100 bilion words from Google News.
* In this part,  we will leave the embeddings fixed instead of updating them (dynamic).

1. __Load `Word2Vec` Pre-trained Word Embedding__

In [19]:
from gensim.models import KeyedVectors
word2vec = KeyedVectors.load_word2vec_format('../GoogleNews-vectors-negative300.bin', binary=True)

In [20]:
# Access the dense vector value for the word 'handsome'
# word2vec.word_vec('handsome') # 0.11376953
word2vec.word_vec('cool') # 1.64062500e-01

array([ 1.64062500e-01,  1.87500000e-01, -4.10156250e-02,  1.25000000e-01,
       -3.22265625e-02,  8.69140625e-02,  1.19140625e-01, -1.26953125e-01,
        1.77001953e-02,  8.83789062e-02,  2.12402344e-02, -2.00195312e-01,
        4.83398438e-02, -1.01074219e-01, -1.89453125e-01,  2.30712891e-02,
        1.17675781e-01,  7.51953125e-02, -8.39843750e-02, -1.33666992e-02,
        1.53320312e-01,  4.08203125e-01,  3.80859375e-02,  3.36914062e-02,
       -4.02832031e-02, -6.88476562e-02,  9.03320312e-02,  2.12890625e-01,
        1.72119141e-02, -6.44531250e-02, -1.29882812e-01,  1.40625000e-01,
        2.38281250e-01,  1.37695312e-01, -1.76757812e-01, -2.71484375e-01,
       -1.36718750e-01, -1.69921875e-01, -9.15527344e-03,  3.47656250e-01,
        2.22656250e-01, -3.06640625e-01,  1.98242188e-01,  1.33789062e-01,
       -4.34570312e-02, -5.12695312e-02, -3.46679688e-02, -8.49609375e-02,
        1.01562500e-01,  1.42578125e-01, -7.95898438e-02,  1.78710938e-01,
        2.30468750e-01,  

2. __Check number of training words present in Word2Vec__

In [21]:
def training_words_in_word2vector(word_to_vec_map, word_to_index):
    '''
    input:
        word_to_vec_map: a word2vec GoogleNews-vectors-negative300.bin model loaded using gensim.models
        word_to_index: word to index mapping from training set
    '''
    
    vocab_size = len(word_to_index) + 1
    count = 0
    # Set each row "idx" of the embedding matrix to be 
    # the word vector representation of the idx'th word of the vocabulary
    for word, idx in word_to_index.items():
        if word in word_to_vec_map:
            count+=1
            
    return print('Found {} words present from {} training vocabulary in the set of pre-trained word vector'.format(count, vocab_size))

In [22]:
# Separate the sentences and the labels
sentences, labels = list(corpus.sentence), list(corpus.label)

# Cleaning and Tokenization
tokenizer = Tokenizer(oov_token=oov_tok)
tokenizer.fit_on_texts(sentences)

word_index = tokenizer.word_index
training_words_in_word2vector(word2vec, word_index)

Found 16448 words present from 18760 training vocabulary in the set of pre-trained word vector


2. __Define a `pretrained_embedding_layer` function__

In [23]:
emb_mean = word2vec.vectors.mean()
emb_std = word2vec.vectors.std()
print('emb_mean: ', emb_mean)
print('emb_std: ', emb_std)

emb_mean:  -0.003527845
emb_std:  0.13315111


In [24]:
from tensorflow.keras.layers import Embedding

def pretrained_embedding_matrix(word_to_vec_map, word_to_index, emb_mean, emb_std):
    '''
    input:
        word_to_vec_map: a word2vec GoogleNews-vectors-negative300.bin model loaded using gensim.models
        word_to_index: word to index mapping from training set
    '''
    np.random.seed(2021)
    
    # adding 1 to fit Keras embedding (requirement)
    vocab_size = len(word_to_index) + 1
    # define dimensionality of your pre-trained word vectors (= 300)
    emb_dim = word_to_vec_map.word_vec('handsome').shape[0]
    
    # initialize the matrix with generic normal distribution values
    embed_matrix = np.random.normal(emb_mean, emb_std, (vocab_size, emb_dim))
    
    # Set each row "idx" of the embedding matrix to be 
    # the word vector representation of the idx'th word of the vocabulary
    for word, idx in word_to_index.items():
        if word in word_to_vec_map:
            embed_matrix[idx] = word_to_vec_map.get_vector(word)
            
    return embed_matrix

In [25]:
w_2_i = {'<UNK>': 1, 'handsome': 2, 'cool': 3, 'shit': 4 }
em_matrix = pretrained_embedding_matrix(word2vec, w_2_i, emb_mean, emb_std)
em_matrix

array([[ 0.19468211,  0.08648376, -0.05924511, ..., -0.16683994,
        -0.09975549, -0.08595189],
       [-0.13509196, -0.07441947,  0.15388953, ..., -0.05400787,
        -0.13156594, -0.05996158],
       [ 0.11376953,  0.1796875 , -0.265625  , ..., -0.21875   ,
        -0.03930664,  0.20996094],
       [ 0.1640625 ,  0.1875    , -0.04101562, ...,  0.10888672,
        -0.01019287,  0.02075195],
       [ 0.10888672, -0.16699219,  0.08984375, ..., -0.19628906,
        -0.23144531,  0.04614258]])

## MLP-CNN-GRU Model

In [26]:
def define_model_2(filters = 100, kernel_size = 3, activation='relu', 
                   input_dim = None, output_dim=300, max_length = None, emb_matrix = None):
  
    # Channel 1
    input1 = Input(shape=(max_length,))
    embeddding1 = Embedding(input_dim=input_dim, 
                            output_dim=output_dim, 
                            input_length=max_length, 
                            input_shape=(max_length, ),
                            # Assign the embedding weight with word2vec embedding marix
                            weights = [emb_matrix],
                            # Set the weight to be not trainable (static)
                            trainable = False)(input1)
    conv1 = Conv1D(filters=filters, kernel_size=kernel_size, activation='relu', 
                   kernel_constraint= MaxNorm( max_value=3, axis=[0,1]))(embeddding1)
    pool1 = MaxPool1D(pool_size=2, strides=2)(conv1)
    flat1 = Flatten()(pool1)
    drop1 = Dropout(0.5)(flat1)
    dense1 = Dense(10, activation='relu')(drop1)
    drop1 = Dropout(0.5)(dense1)
    out1 = Dense(1, activation='sigmoid')(drop1)
    
    # Channel 2
    input2 = Input(shape=(max_length,))
    embeddding2 = Embedding(input_dim=input_dim, 
                            output_dim=output_dim, 
                            input_length=max_length, 
                            input_shape=(max_length, ),
                            # Assign the embedding weight with word2vec embedding marix
                            weights = [emb_matrix],
                            # Set the weight to be not trainable (static)
                            trainable = False,
                            mask_zero=True)(input2)
    gru2 = Bidirectional(GRU(64))(embeddding2)
    drop2 = Dropout(0.5)(gru2)
    out2 = Dense(1, activation='sigmoid')(drop2)
    
    # Merge
    merged = concatenate([out1, out2])
    
    # Interpretation
    outputs = Dense(1, activation='sigmoid')(merged)
    model = Model(inputs=[input1, input2], outputs=outputs)
    
    # Compile
    model.compile( loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    
    return model

In [27]:
model_0 = define_model_2( input_dim=1000, max_length=100, emb_matrix=np.random.rand(1000, 300))
model_0.summary()

Model: "model_61"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_123 (InputLayer)          [(None, 100)]        0                                            
__________________________________________________________________________________________________
embedding_122 (Embedding)       (None, 100, 300)     300000      input_123[0][0]                  
__________________________________________________________________________________________________
conv1d_61 (Conv1D)              (None, 98, 100)      90100       embedding_122[0][0]              
__________________________________________________________________________________________________
max_pooling1d_61 (MaxPooling1D) (None, 49, 100)      0           conv1d_61[0][0]                  
___________________________________________________________________________________________

## Train and Test the Model

In [28]:
class myCallback(tf.keras.callbacks.Callback):
    # Overide the method on_epoch_end() for our benefit
    def on_epoch_end(self, epoch, logs={}):
        if (logs.get('accuracy') >= 0.9):
            print("\nReached 90% accuracy so cancelling training!")
            self.model.stop_training=True

callbacks = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', min_delta=0, 
                                             patience=7, verbose=2, 
                                             mode='auto', restore_best_weights=True)

In [29]:
# Parameter Initialization
trunc_type='post'
padding_type='post'
oov_tok = "<UNK>"
activations = ['relu']
filters = 100
kernel_sizes = [1, 2, 3, 4, 5, 6, 7, 8]
emb_mean = emb_mean
emb_std = emb_std

columns = ['Activation', 'Filters', 'acc1', 'acc2', 'acc3', 'acc4', 'acc5', 'acc6', 'acc7', 'acc8', 'acc9', 'acc10', 'AVG']
record2 = pd.DataFrame(columns = columns)

# prepare cross validation with 10 splits and shuffle = True
kfold = KFold(10, True)

# Separate the sentences and the labels
sentences, labels = list(corpus.sentence), list(corpus.label)

for activation in activations:
    for kernel_size in kernel_sizes:
        # kfold.split() will return set indices for each split
        acc_list = []
        for train, test in kfold.split(sentences):
            
            train_x, test_x = [], []
            train_y, test_y = [], []
            
            for i in train:
                train_x.append(sentences[i])
                train_y.append(labels[i])

            for i in test:
                test_x.append(sentences[i])
                test_y.append(labels[i])

            # Turn the labels into a numpy array
            train_y = np.array(train_y)
            test_y = np.array(test_y)

            # encode data using
            # Cleaning and Tokenization
            tokenizer = Tokenizer(oov_token=oov_tok)
            tokenizer.fit_on_texts(train_x)

            # Turn the text into sequence
            training_sequences = tokenizer.texts_to_sequences(train_x)
            test_sequences = tokenizer.texts_to_sequences(test_x)

            max_len = max_length(training_sequences)

            # Pad the sequence to have the same size
            Xtrain = pad_sequences(training_sequences, maxlen=max_len, padding=padding_type, truncating=trunc_type)
            Xtest = pad_sequences(test_sequences, maxlen=max_len, padding=padding_type, truncating=trunc_type)

            word_index = tokenizer.word_index
            vocab_size = len(word_index)+1
            
            
            emb_matrix = pretrained_embedding_matrix(word2vec, word_index, emb_mean, emb_std)
            
            # Define the input shape
            model = define_model_2(filters, kernel_size, activation, input_dim=vocab_size, 
                                 max_length=max_len, emb_matrix=emb_matrix)

            # Train the model and initialize test accuracy with 0
            acc = 0
            while(acc<0.7):
                
                # Train the model
                model.fit(x=[Xtrain, Xtrain], y = train_y, batch_size=50, epochs=100, verbose=1, 
                          callbacks=[callbacks], validation_data=([Xtest, Xtest], test_y))

                # evaluate the model
                loss, acc = model.evaluate([Xtest, Xtest], test_y, verbose=0)
                print('Test Accuracy: {}'.format(acc*100))
                
                if (acc<0.7):
                    print('The model suffered from local minimum. Retrain the model!')
                    model = define_model_2(filters, kernel_size, activation, input_dim=vocab_size, 
                                           max_length=max_len, emb_matrix=emb_matrix)
                    
                else:
                    print('Done!')

            acc_list.append(acc*100)
            
        mean_acc = np.array(acc_list).mean()
        parameters = [activation, kernel_size]
        entries = parameters + acc_list + [mean_acc]

        temp = pd.DataFrame([entries], columns=columns)
        record2 = record2.append(temp, ignore_index=True)
        print()
        print(record2)
        print()

C:\Users\Diardano Raihan\Anaconda3\envs\tf-gpu\lib\site-packages\sklearn\utils\validation.py:72: FutureWarning: Pass shuffle=True as keyword args. From version 1.0 (renaming of 0.25) passing these as positional arguments will result in an error
  "will result in an error", FutureWarning)


Epoch 1/100
192/192 [==============================] - 33s 100ms/step - loss: 0.6895 - accuracy: 0.5065 - val_loss: 0.6701 - val_accuracy: 0.4714
Epoch 2/100
192/192 [==============================] - 13s 68ms/step - loss: 0.6573 - accuracy: 0.6232 - val_loss: 0.6415 - val_accuracy: 0.7432
Epoch 3/100
192/192 [==============================] - 13s 68ms/step - loss: 0.6258 - accuracy: 0.7665 - val_loss: 0.6183 - val_accuracy: 0.7432
Epoch 4/100
192/192 [==============================] - 14s 71ms/step - loss: 0.5987 - accuracy: 0.7692 - val_loss: 0.5772 - val_accuracy: 0.7769
Epoch 5/100
192/192 [==============================] - 13s 70ms/step - loss: 0.5664 - accuracy: 0.7899 - val_loss: 0.5492 - val_accuracy: 0.7863
Epoch 6/100
192/192 [==============================] - 14s 72ms/step - loss: 0.5386 - accuracy: 0.7943 - val_loss: 0.5257 - val_accuracy: 0.8022
Epoch 7/100
192/192 [==============================] - 13s 67ms/step - loss: 0.5206 - accuracy: 0.7972 - val_loss: 0.5179 - val_a

Epoch 23/100
192/192 [==============================] - 17s 87ms/step - loss: 0.2456 - accuracy: 0.9383 - val_loss: 0.5008 - val_accuracy: 0.7992
Epoch 24/100
192/192 [==============================] - 17s 87ms/step - loss: 0.2256 - accuracy: 0.9451 - val_loss: 0.5239 - val_accuracy: 0.7899
Epoch 25/100
192/192 [==============================] - 16s 83ms/step - loss: 0.2210 - accuracy: 0.9471 - val_loss: 0.5353 - val_accuracy: 0.7805
Epoch 26/100
192/192 [==============================] - 14s 74ms/step - loss: 0.2237 - accuracy: 0.9424 - val_loss: 0.5448 - val_accuracy: 0.7786
Epoch 27/100
192/192 [==============================] - 14s 71ms/step - loss: 0.2012 - accuracy: 0.9526 - val_loss: 0.5376 - val_accuracy: 0.7833
Epoch 28/100
192/192 [==============================] - 14s 74ms/step - loss: 0.1989 - accuracy: 0.9529 - val_loss: 0.5522 - val_accuracy: 0.7861
Epoch 29/100
192/192 [==============================] - 15s 78ms/step - loss: 0.2027 - accuracy: 0.9502 - val_loss: 0.5412 -

192/192 [==============================] - 17s 88ms/step - loss: 0.3862 - accuracy: 0.8880 - val_loss: 0.4967 - val_accuracy: 0.7927
Restoring model weights from the end of the best epoch.
Epoch 00013: early stopping
Test Accuracy: 79.83114719390869
Done!
Epoch 1/100
192/192 [==============================] - 37s 96ms/step - loss: 0.6722 - accuracy: 0.6165 - val_loss: 0.6191 - val_accuracy: 0.7214
Epoch 2/100
192/192 [==============================] - 13s 70ms/step - loss: 0.5980 - accuracy: 0.7504 - val_loss: 0.5730 - val_accuracy: 0.7270
Epoch 3/100
192/192 [==============================] - 13s 69ms/step - loss: 0.5698 - accuracy: 0.7490 - val_loss: 0.5325 - val_accuracy: 0.7683
Epoch 4/100
192/192 [==============================] - 14s 73ms/step - loss: 0.5223 - accuracy: 0.7892 - val_loss: 0.5214 - val_accuracy: 0.7683
Epoch 5/100
192/192 [==============================] - 13s 70ms/step - loss: 0.5021 - accuracy: 0.7848 - val_loss: 0.4950 - val_accuracy: 0.7786
Epoch 6/100
192/192

Epoch 6/100
192/192 [==============================] - 13s 70ms/step - loss: 0.5153 - accuracy: 0.8169 - val_loss: 0.5142 - val_accuracy: 0.8039
Epoch 7/100
192/192 [==============================] - 14s 75ms/step - loss: 0.4955 - accuracy: 0.8258 - val_loss: 0.5131 - val_accuracy: 0.8002
Epoch 8/100
192/192 [==============================] - 13s 69ms/step - loss: 0.4870 - accuracy: 0.8226 - val_loss: 0.4998 - val_accuracy: 0.8021
Epoch 9/100
192/192 [==============================] - 14s 70ms/step - loss: 0.4589 - accuracy: 0.8434 - val_loss: 0.4934 - val_accuracy: 0.7974
Epoch 10/100
192/192 [==============================] - 14s 71ms/step - loss: 0.4468 - accuracy: 0.8481 - val_loss: 0.4903 - val_accuracy: 0.7889
Epoch 11/100
192/192 [==============================] - 13s 70ms/step - loss: 0.4136 - accuracy: 0.8694 - val_loss: 0.4885 - val_accuracy: 0.7946
Epoch 12/100
192/192 [==============================] - 14s 71ms/step - loss: 0.3980 - accuracy: 0.8739 - val_loss: 0.4827 - val

...

Epoch 1/100
192/192 [==============================] - 32s 91ms/step - loss: 0.6669 - accuracy: 0.6117 - val_loss: 0.5972 - val_accuracy: 0.7523
Epoch 2/100
192/192 [==============================] - 13s 70ms/step - loss: 0.5878 - accuracy: 0.7579 - val_loss: 0.5773 - val_accuracy: 0.7645
Epoch 3/100
192/192 [==============================] - 13s 70ms/step - loss: 0.5599 - accuracy: 0.7855 - val_loss: 0.5605 - val_accuracy: 0.7683
Epoch 4/100
192/192 [==============================] - 13s 69ms/step - loss: 0.5343 - accuracy: 0.8028 - val_loss: 0.5458 - val_accuracy: 0.7777
Epoch 5/100
192/192 [==============================] - 13s 69ms/step - loss: 0.5150 - accuracy: 0.8117 - val_loss: 0.5351 - val_accuracy: 0.7786
Epoch 6/100
192/192 [==============================] - 13s 70ms/step - loss: 0.4945 - accuracy: 0.8227 - val_loss: 0.5194 - val_accuracy: 0.7852
Epoch 7/100
192/192 [==============================] - 13s 69ms/step - loss: 0.4705 - accuracy: 0.8415 - val_loss: 0.5511 - val_ac

Epoch 8/100
192/192 [==============================] - 14s 75ms/step - loss: 0.5254 - accuracy: 0.8160 - val_loss: 0.5306 - val_accuracy: 0.7927
Epoch 9/100
192/192 [==============================] - 15s 76ms/step - loss: 0.5056 - accuracy: 0.8311 - val_loss: 0.5210 - val_accuracy: 0.8021
Epoch 10/100
192/192 [==============================] - 15s 76ms/step - loss: 0.4836 - accuracy: 0.8427 - val_loss: 0.5092 - val_accuracy: 0.8049
Epoch 11/100
192/192 [==============================] - 14s 75ms/step - loss: 0.4644 - accuracy: 0.8520 - val_loss: 0.4983 - val_accuracy: 0.8105
Epoch 12/100
192/192 [==============================] - 14s 75ms/step - loss: 0.4480 - accuracy: 0.8588 - val_loss: 0.4946 - val_accuracy: 0.8021
Epoch 13/100
192/192 [==============================] - 15s 76ms/step - loss: 0.4288 - accuracy: 0.8709 - val_loss: 0.5125 - val_accuracy: 0.7814
Epoch 14/100
192/192 [==============================] - 14s 75ms/step - loss: 0.4136 - accuracy: 0.8787 - val_loss: 0.4993 - v

192/192 [==============================] - 14s 71ms/step - loss: 0.5347 - accuracy: 0.7876 - val_loss: 0.5352 - val_accuracy: 0.7805
Epoch 5/100
192/192 [==============================] - 13s 70ms/step - loss: 0.5087 - accuracy: 0.8134 - val_loss: 0.5389 - val_accuracy: 0.7664
Epoch 6/100
192/192 [==============================] - 14s 71ms/step - loss: 0.4913 - accuracy: 0.8191 - val_loss: 0.5188 - val_accuracy: 0.7805
Epoch 7/100
192/192 [==============================] - 14s 75ms/step - loss: 0.4646 - accuracy: 0.8408 - val_loss: 0.5342 - val_accuracy: 0.7495
Epoch 8/100
192/192 [==============================] - 14s 71ms/step - loss: 0.4481 - accuracy: 0.8462 - val_loss: 0.5141 - val_accuracy: 0.7702
Epoch 9/100
192/192 [==============================] - 13s 70ms/step - loss: 0.4192 - accuracy: 0.8638 - val_loss: 0.5159 - val_accuracy: 0.7758
Epoch 10/100
192/192 [==============================] - 14s 70ms/step - loss: 0.4001 - accuracy: 0.8733 - val_loss: 0.5163 - val_accuracy: 0.7

Epoch 6/100
192/192 [==============================] - 15s 80ms/step - loss: 0.5108 - accuracy: 0.8163 - val_loss: 0.5222 - val_accuracy: 0.7730
Epoch 7/100
192/192 [==============================] - 17s 87ms/step - loss: 0.4825 - accuracy: 0.8285 - val_loss: 0.5061 - val_accuracy: 0.7805
Epoch 8/100
192/192 [==============================] - 16s 84ms/step - loss: 0.4604 - accuracy: 0.8323 - val_loss: 0.5007 - val_accuracy: 0.7805
Epoch 9/100
192/192 [==============================] - 16s 84ms/step - loss: 0.4273 - accuracy: 0.8510 - val_loss: 0.5033 - val_accuracy: 0.7720
Epoch 10/100
192/192 [==============================] - 16s 82ms/step - loss: 0.4028 - accuracy: 0.8540 - val_loss: 0.4954 - val_accuracy: 0.7720
Epoch 11/100
192/192 [==============================] - 16s 85ms/step - loss: 0.3743 - accuracy: 0.8752 - val_loss: 0.4940 - val_accuracy: 0.7674
Epoch 12/100
192/192 [==============================] - 15s 80ms/step - loss: 0.3538 - accuracy: 0.8796 - val_loss: 0.5036 - val

KeyboardInterrupt: 

## Summary

In [30]:
record2.sort_values(by='AVG', ascending=False)

Activation Filters       acc1       acc2       acc3       acc4       acc5  \
1       relu       2  79.568887  80.318648  78.236395  78.799248  79.549718   
2       relu       3  80.318648  78.725398  80.581611  79.080677  78.330207   
3       relu       4  78.537959  80.693531  78.424013  79.831147  80.112571   
0       relu       1  80.224931  77.881914  77.579737  79.924953  80.206376   
5       relu       6  78.912842  78.444237  80.487806  79.549718  78.799248   
4       relu       5  79.194003  78.912842  78.705442  80.393994  80.018759   

        acc6       acc7       acc8       acc9      acc10        AVG  
1  81.144464  79.924953  81.332082  79.268295  79.924953  79.806764  
2  78.986865  80.112571  78.424013  78.986865  81.144464  79.469132  
3  79.268295  79.643530  79.549718  78.048778  79.737335  79.384688  
0  78.893059  79.831147  79.643530  78.330207  80.393994  79.290985  
5  78.611630  81.050658  79.362100  79.362100  78.048778  79.262912  
4  80.581611  79.080677  78.142589  78.142589  78.048778  79.122128

In [31]:
record2[['Activation', 'AVG']].groupby(by='Activation').max().sort_values(by='AVG', ascending=False)

AVG
Activation           
relu        79.806764

In [32]:
report = record2.sort_values(by='AVG', ascending=False)
report = report.to_excel('HYBRID_MR_2.xlsx', sheet_name='static')

# Model 3: Word2Vec - Dynamic

* In this part,  we will fine tune the embeddings while training (dynamic).

## MLP-CNN-GRU Model

In [33]:
def define_model_3(filters = 100, kernel_size = 3, activation='relu', 
                   input_dim = None, output_dim=300, max_length = None, emb_matrix = None):
  
    # Channel 1
    input1 = Input(shape=(max_length,))
    embeddding1 = Embedding(input_dim=input_dim, 
                            output_dim=output_dim, 
                            input_length=max_length, 
                            input_shape=(max_length, ),
                            # Assign the embedding weight with word2vec embedding marix
                            weights = [emb_matrix],
                            # Set the weight to be not trainable (static)
                            trainable = True)(input1)
    conv1 = Conv1D(filters=filters, kernel_size=kernel_size, activation='relu', 
                   kernel_constraint= MaxNorm( max_value=3, axis=[0,1]))(embeddding1)
    pool1 = MaxPool1D(pool_size=2, strides=2)(conv1)
    flat1 = Flatten()(pool1)
    drop1 = Dropout(0.5)(flat1)
    dense1 = Dense(10, activation='relu')(drop1)
    drop1 = Dropout(0.5)(dense1)
    out1 = Dense(1, activation='sigmoid')(drop1)
    
    # Channel 2
    input2 = Input(shape=(max_length,))
    embeddding2 = Embedding(input_dim=input_dim, 
                            output_dim=output_dim, 
                            input_length=max_length, 
                            input_shape=(max_length, ),
                            # Assign the embedding weight with word2vec embedding marix
                            weights = [emb_matrix],
                            # Set the weight to be not trainable (static)
                            trainable = True,
                            mask_zero=True)(input2)
    gru2 = Bidirectional(GRU(64))(embeddding2)
    drop2 = Dropout(0.5)(gru2)
    out2 = Dense(1, activation='sigmoid')(drop2)
    
    # Merge
    merged = concatenate([out1, out2])
    
    # Interpretation
    outputs = Dense(1, activation='sigmoid')(merged)
    model = Model(inputs=[input1, input2], outputs=outputs)
    
    # Compile
    model.compile( loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    
    return model

In [34]:
model_0 = define_model_3( input_dim=1000, max_length=100, emb_matrix=np.random.rand(1000, 300))
model_0.summary()

Model: "model_126"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_253 (InputLayer)          [(None, 100)]        0                                            
__________________________________________________________________________________________________
embedding_252 (Embedding)       (None, 100, 300)     300000      input_253[0][0]                  
__________________________________________________________________________________________________
conv1d_126 (Conv1D)             (None, 98, 100)      90100       embedding_252[0][0]              
__________________________________________________________________________________________________
max_pooling1d_126 (MaxPooling1D (None, 49, 100)      0           conv1d_126[0][0]                 
__________________________________________________________________________________________

## Train and Test the Model

In [35]:
class myCallback(tf.keras.callbacks.Callback):
    # Overide the method on_epoch_end() for our benefit
    def on_epoch_end(self, epoch, logs={}):
        if (logs.get('accuracy') > 0.93):
            print("\nReached 93% accuracy so cancelling training!")
            self.model.stop_training=True

callbacks = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', min_delta=0, 
                                             patience=7, verbose=2, 
                                             mode='auto', restore_best_weights=True)

In [36]:
# Parameter Initialization
trunc_type='post'
padding_type='post'
oov_tok = "<UNK>"
activations = ['relu']
filters = 100
kernel_sizes = [1, 2, 3, 4, 5, 6]
emb_mean = emb_mean
emb_std = emb_std

columns = ['Activation', 'Filters', 'acc1', 'acc2', 'acc3', 'acc4', 'acc5', 'acc6', 'acc7', 'acc8', 'acc9', 'acc10', 'AVG']
record3 = pd.DataFrame(columns = columns)

# prepare cross validation with 10 splits and shuffle = True
kfold = KFold(10, True)

# Separate the sentences and the labels
sentences, labels = list(corpus.sentence), list(corpus.label)

for activation in activations:
    for kernel_size in kernel_sizes:
        # kfold.split() will return set indices for each split
        acc_list = []
        for train, test in kfold.split(sentences):
            
            train_x, test_x = [], []
            train_y, test_y = [], []
            
            for i in train:
                train_x.append(sentences[i])
                train_y.append(labels[i])

            for i in test:
                test_x.append(sentences[i])
                test_y.append(labels[i])

            # Turn the labels into a numpy array
            train_y = np.array(train_y)
            test_y = np.array(test_y)

            # encode data using
            # Cleaning and Tokenization
            tokenizer = Tokenizer(oov_token=oov_tok)
            tokenizer.fit_on_texts(train_x)

            # Turn the text into sequence
            training_sequences = tokenizer.texts_to_sequences(train_x)
            test_sequences = tokenizer.texts_to_sequences(test_x)

            max_len = max_length(training_sequences)

            # Pad the sequence to have the same size
            Xtrain = pad_sequences(training_sequences, maxlen=max_len, padding=padding_type, truncating=trunc_type)
            Xtest = pad_sequences(test_sequences, maxlen=max_len, padding=padding_type, truncating=trunc_type)

            word_index = tokenizer.word_index
            vocab_size = len(word_index)+1
            
            
            emb_matrix = pretrained_embedding_matrix(word2vec, word_index, emb_mean, emb_std)
            
            # Define the input shape
            model = define_model_3(filters, kernel_size, activation, input_dim=vocab_size, 
                                 max_length=max_len, emb_matrix=emb_matrix)

            # Train the model and initialize test accuracy with 0
            acc = 0
            while(acc<0.7):
                
                # Train the model
                model.fit(x=[Xtrain, Xtrain], y = train_y, batch_size=50, epochs=100, verbose=1, 
                          callbacks=[callbacks], validation_data=([Xtest, Xtest], test_y))

                # evaluate the model
                loss, acc = model.evaluate([Xtest, Xtest], test_y, verbose=0)
                print('Test Accuracy: {}'.format(acc*100))
                
                if (acc<0.7):
                    print('The model suffered from local minimum. Retrain the model!')
                    model = define_model_3(filters, kernel_size, activation, input_dim=vocab_size, 
                                           max_length=max_len, emb_matrix=emb_matrix)
                    
                else:
                    print('Done!')
            acc_list.append(acc*100)
            
        mean_acc = np.array(acc_list).mean()
        parameters = [activation, kernel_size]
        entries = parameters + acc_list + [mean_acc]

        temp = pd.DataFrame([entries], columns=columns)
        record3 = record3.append(temp, ignore_index=True)
        print()
        print(record3)
        print()

C:\Users\Diardano Raihan\Anaconda3\envs\tf-gpu\lib\site-packages\sklearn\utils\validation.py:72: FutureWarning: Pass shuffle=True as keyword args. From version 1.0 (renaming of 0.25) passing these as positional arguments will result in an error
  "will result in an error", FutureWarning)


Epoch 1/100
192/192 [==============================] - 62s 239ms/step - loss: 0.6437 - accuracy: 0.6340 - val_loss: 0.5384 - val_accuracy: 0.7638
Epoch 2/100
192/192 [==============================] - 36s 189ms/step - loss: 0.4635 - accuracy: 0.8584 - val_loss: 0.4824 - val_accuracy: 0.7891
Epoch 3/100
192/192 [==============================] - 38s 200ms/step - loss: 0.3444 - accuracy: 0.9155 - val_loss: 0.4689 - val_accuracy: 0.7938
Epoch 4/100
192/192 [==============================] - 39s 205ms/step - loss: 0.2677 - accuracy: 0.9478 - val_loss: 0.4665 - val_accuracy: 0.7985
Epoch 5/100
192/192 [==============================] - 39s 203ms/step - loss: 0.2222 - accuracy: 0.9597 - val_loss: 0.4924 - val_accuracy: 0.7854
Epoch 6/100
192/192 [==============================] - 37s 192ms/step - loss: 0.1849 - accuracy: 0.9693 - val_loss: 0.4816 - val_accuracy: 0.7919
Epoch 7/100
192/192 [==============================] - 38s 196ms/step - loss: 0.1556 - accuracy: 0.9748 - val_loss: 0.5013 -

192/192 [==============================] - 40s 211ms/step - loss: 0.2342 - accuracy: 0.9772 - val_loss: 0.5198 - val_accuracy: 0.7758
Epoch 14/100
192/192 [==============================] - 40s 209ms/step - loss: 0.2157 - accuracy: 0.9816 - val_loss: 0.5113 - val_accuracy: 0.7861
Epoch 15/100
192/192 [==============================] - 40s 209ms/step - loss: 0.2084 - accuracy: 0.9795 - val_loss: 0.5255 - val_accuracy: 0.7730
Restoring model weights from the end of the best epoch.
Epoch 00015: early stopping
Test Accuracy: 79.17448282241821
Done!
Epoch 1/100
192/192 [==============================] - 60s 232ms/step - loss: 0.6759 - accuracy: 0.5868 - val_loss: 0.5898 - val_accuracy: 0.7561
Epoch 2/100
192/192 [==============================] - 40s 209ms/step - loss: 0.5303 - accuracy: 0.8168 - val_loss: 0.4963 - val_accuracy: 0.7992
Epoch 3/100
192/192 [==============================] - 39s 206ms/step - loss: 0.4001 - accuracy: 0.8966 - val_loss: 0.4888 - val_accuracy: 0.7833
Epoch 4/100

Epoch 4/100
192/192 [==============================] - 40s 207ms/step - loss: 0.4678 - accuracy: 0.9098 - val_loss: 0.5552 - val_accuracy: 0.7730
Epoch 5/100
192/192 [==============================] - 40s 207ms/step - loss: 0.4218 - accuracy: 0.9463 - val_loss: 0.5417 - val_accuracy: 0.7833
Epoch 6/100
192/192 [==============================] - 40s 207ms/step - loss: 0.3864 - accuracy: 0.9552 - val_loss: 0.5279 - val_accuracy: 0.7889
Epoch 7/100
192/192 [==============================] - 41s 212ms/step - loss: 0.3526 - accuracy: 0.9649 - val_loss: 0.5224 - val_accuracy: 0.7824
Epoch 8/100
192/192 [==============================] - 40s 209ms/step - loss: 0.3284 - accuracy: 0.9671 - val_loss: 0.5260 - val_accuracy: 0.7749
Epoch 9/100
192/192 [==============================] - 41s 215ms/step - loss: 0.3066 - accuracy: 0.9701 - val_loss: 0.5236 - val_accuracy: 0.7720
Epoch 10/100
192/192 [==============================] - 41s 215ms/step - loss: 0.2887 - accuracy: 0.9694 - val_loss: 0.5291 

192/192 [==============================] - 40s 209ms/step - loss: 0.3145 - accuracy: 0.9721 - val_loss: 0.5282 - val_accuracy: 0.7795
Epoch 11/100
192/192 [==============================] - 40s 210ms/step - loss: 0.2950 - accuracy: 0.9760 - val_loss: 0.5381 - val_accuracy: 0.7608
Epoch 12/100
192/192 [==============================] - 40s 210ms/step - loss: 0.2779 - accuracy: 0.9757 - val_loss: 0.5342 - val_accuracy: 0.7683
Restoring model weights from the end of the best epoch.
Epoch 00012: early stopping
Test Accuracy: 78.23639512062073
Done!
Epoch 1/100
192/192 [==============================] - 59s 231ms/step - loss: 0.6797 - accuracy: 0.5678 - val_loss: 0.6297 - val_accuracy: 0.7495
Epoch 2/100
192/192 [==============================] - 40s 208ms/step - loss: 0.5930 - accuracy: 0.8448 - val_loss: 0.6039 - val_accuracy: 0.7617
Epoch 3/100
192/192 [==============================] - 40s 209ms/step - loss: 0.5423 - accuracy: 0.8976 - val_loss: 0.5868 - val_accuracy: 0.7598
Epoch 4/100

...

Epoch 7/100
192/192 [==============================] - 41s 213ms/step - loss: 0.2972 - accuracy: 0.9614 - val_loss: 0.4738 - val_accuracy: 0.7842
Epoch 8/100
192/192 [==============================] - 42s 219ms/step - loss: 0.2661 - accuracy: 0.9623 - val_loss: 0.4788 - val_accuracy: 0.7842
Epoch 9/100
192/192 [==============================] - 41s 216ms/step - loss: 0.2374 - accuracy: 0.9672 - val_loss: 0.4836 - val_accuracy: 0.7908
Epoch 10/100
192/192 [==============================] - 41s 214ms/step - loss: 0.2167 - accuracy: 0.9700 - val_loss: 0.4870 - val_accuracy: 0.7899
Epoch 11/100
192/192 [==============================] - 41s 214ms/step - loss: 0.1960 - accuracy: 0.9703 - val_loss: 0.4839 - val_accuracy: 0.7927
Restoring model weights from the end of the best epoch.
Epoch 00011: early stopping
Test Accuracy: 80.48780560493469
Done!
Epoch 1/100
192/192 [==============================] - 112s 247ms/step - loss: 0.6872 - accuracy: 0.5617 - val_loss: 0.6399 - val_accuracy: 0.740

Test Accuracy: 77.97563076019287
Done!
Epoch 1/100
192/192 [==============================] - 61s 237ms/step - loss: 0.6548 - accuracy: 0.6218 - val_loss: 0.5365 - val_accuracy: 0.7927
Epoch 2/100
192/192 [==============================] - 41s 214ms/step - loss: 0.4742 - accuracy: 0.8481 - val_loss: 0.4879 - val_accuracy: 0.7992
Epoch 3/100
192/192 [==============================] - 41s 215ms/step - loss: 0.3561 - accuracy: 0.9140 - val_loss: 0.4941 - val_accuracy: 0.7786
Epoch 4/100
192/192 [==============================] - 42s 217ms/step - loss: 0.2893 - accuracy: 0.9383 - val_loss: 0.4966 - val_accuracy: 0.7786
Epoch 5/100
192/192 [==============================] - 42s 217ms/step - loss: 0.2312 - accuracy: 0.9589 - val_loss: 0.4988 - val_accuracy: 0.7758
Epoch 6/100
192/192 [==============================] - 43s 223ms/step - loss: 0.1959 - accuracy: 0.9704 - val_loss: 0.5159 - val_accuracy: 0.7739
Epoch 7/100
192/192 [==============================] - 36s 186ms/step - loss: 0.1702 

192/192 [==============================] - 36s 186ms/step - loss: 0.4116 - accuracy: 0.9317 - val_loss: 0.5372 - val_accuracy: 0.7777
Epoch 5/100
192/192 [==============================] - 36s 186ms/step - loss: 0.3613 - accuracy: 0.9574 - val_loss: 0.5311 - val_accuracy: 0.7795
Epoch 6/100
192/192 [==============================] - 36s 188ms/step - loss: 0.3254 - accuracy: 0.9651 - val_loss: 0.5229 - val_accuracy: 0.7805
Epoch 7/100
192/192 [==============================] - 36s 185ms/step - loss: 0.2983 - accuracy: 0.9715 - val_loss: 0.5152 - val_accuracy: 0.7833
Epoch 8/100
192/192 [==============================] - 36s 188ms/step - loss: 0.2788 - accuracy: 0.9699 - val_loss: 0.5391 - val_accuracy: 0.7683
Epoch 9/100
192/192 [==============================] - 35s 185ms/step - loss: 0.2534 - accuracy: 0.9768 - val_loss: 0.5326 - val_accuracy: 0.7674
Epoch 10/100
192/192 [==============================] - 36s 186ms/step - loss: 0.2363 - accuracy: 0.9745 - val_loss: 0.5785 - val_accura

192/192 [==============================] - 36s 188ms/step - loss: 0.2350 - accuracy: 0.9577 - val_loss: 0.4663 - val_accuracy: 0.7910
Epoch 9/100
192/192 [==============================] - 36s 186ms/step - loss: 0.2041 - accuracy: 0.9686 - val_loss: 0.4565 - val_accuracy: 0.8088
Epoch 10/100
192/192 [==============================] - 36s 185ms/step - loss: 0.1883 - accuracy: 0.9691 - val_loss: 0.4531 - val_accuracy: 0.8219
Epoch 11/100
192/192 [==============================] - 36s 187ms/step - loss: 0.1694 - accuracy: 0.9735 - val_loss: 0.4595 - val_accuracy: 0.8022
Epoch 12/100
192/192 [==============================] - 36s 188ms/step - loss: 0.1581 - accuracy: 0.9760 - val_loss: 0.4801 - val_accuracy: 0.7948
Epoch 13/100
192/192 [==============================] - 36s 187ms/step - loss: 0.1430 - accuracy: 0.9765 - val_loss: 0.4853 - val_accuracy: 0.8032
Epoch 14/100
192/192 [==============================] - 36s 190ms/step - loss: 0.1345 - accuracy: 0.9772 - val_loss: 0.4890 - val_ac

Epoch 2/100
192/192 [==============================] - 36s 186ms/step - loss: 0.4880 - accuracy: 0.8289 - val_loss: 0.4577 - val_accuracy: 0.8049
Epoch 3/100
192/192 [==============================] - 36s 186ms/step - loss: 0.3567 - accuracy: 0.9025 - val_loss: 0.4459 - val_accuracy: 0.8124
Epoch 4/100
192/192 [==============================] - 36s 185ms/step - loss: 0.2878 - accuracy: 0.9366 - val_loss: 0.4517 - val_accuracy: 0.8030
Epoch 5/100
192/192 [==============================] - 36s 190ms/step - loss: 0.2268 - accuracy: 0.9619 - val_loss: 0.4407 - val_accuracy: 0.8086
Epoch 6/100
192/192 [==============================] - 37s 191ms/step - loss: 0.1928 - accuracy: 0.9648 - val_loss: 0.4437 - val_accuracy: 0.8114
Epoch 7/100
192/192 [==============================] - 38s 197ms/step - loss: 0.1705 - accuracy: 0.9708 - val_loss: 0.4492 - val_accuracy: 0.8049
Epoch 8/100
192/192 [==============================] - 40s 207ms/step - loss: 0.1528 - accuracy: 0.9730 - val_loss: 0.4755 -

## Summary

In [37]:
record3.sort_values(by='AVG', ascending=False)

Activation Filters       acc1       acc2       acc3       acc4       acc5  \
1       relu       2  79.381442  79.568887  80.675423  78.236395  76.735461   
2       relu       3  80.037487  78.631675  78.611630  80.487806  78.893059   
0       relu       1  79.850048  78.631675  78.986865  78.705442  79.174483   
4       relu       5  77.507031  77.975631  79.924953  79.455912  80.206376   
5       relu       6  78.537959  82.193065  77.767354  76.735461  77.579737   
3       relu       4  78.069353  79.475164  77.016884  79.362100  80.300188   

        acc6       acc7       acc8       acc9      acc10        AVG  
1  79.174483  79.362100  79.455912  81.988746  79.737335  79.431618  
2  79.455912  78.330207  78.424013  80.206376  80.581611  79.365978  
0  79.924953  79.643530  79.268295  79.268295  78.893059  79.234664  
4  80.112571  79.268295  78.330207  77.579737  80.206376  79.056709  
5  79.362100  81.238276  79.268295  78.236395  77.673548  78.859219  
3  78.986865  78.236395  80.487806  76.829267  79.080677  78.784470

In [38]:
report = record3.sort_values(by='AVG', ascending=False)
report = report.to_excel('HYBRID_MR_3.xlsx', sheet_name='dynamic')